In [62]:
%matplotlib inline
import pylab as pl
import numpy as np
import pandas as pd

#Read data

X = []
samples_x = []

f = open("genotypes.csv","r")

for i,line in enumerate(f):
    if(i == 0):
       continue
    
    sv = line.strip().split(",")
    samples_x.append(sv[0])
    
    row = []
    for element in sv[1:]:
        row.append(element)
    X.append(row)
    
f.close()

X = np.array(X)
samples_x = np.array(samples_x)

#print(samples_x)
#print(X)


[['G' 'G' 'G' ... '?' 'T' 'A']
 ['G' 'G' 'G' ... 'A' 'T' 'A']
 ['A' 'G' 'A' ... 'A' '?' 'A']
 ...
 ['G' 'A' 'G' ... 'T' 'T' 'A']
 ['G' 'G' 'A' ... 'A' 'T' 'A']
 ['G' 'G' 'G' ... 'A' 'T' 'C']]


In [63]:
#Replace "?" with np.nan

X = np.where(X == "?", np.nan, X)

#print(X)

[['G' 'G' 'G' ... 'nan' 'T' 'A']
 ['G' 'G' 'G' ... 'A' 'T' 'A']
 ['A' 'G' 'A' ... 'A' 'nan' 'A']
 ...
 ['G' 'A' 'G' ... 'T' 'T' 'A']
 ['G' 'G' 'A' ... 'A' 'T' 'A']
 ['G' 'G' 'G' ... 'A' 'T' 'C']]


In [67]:
#Encode nucleotide values

for j in range (X.shape[1]):
    
    values, counts = np.unique(X[:,j], return_counts=True)
    values=values[0:2]
    counts=counts[0:2]

    #print(values[np.argmax(counts)])
    #print(values[np.argmin(counts)])
    #print(values, counts)

    X[:,j] = np.where(X[:,j] == values[np.argmax(counts)], 0.0, X[:,j])
    X[:,j] = np.where(X[:,j] == values[np.argmin(counts)], 2.0, X[:,j])

#Convert array from string to float
X = X.astype(np.float_)

#print(X)
    

[[ 0.  0.  0. ... nan  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 2.  0.  2. ...  0. nan  0.]
 ...
 [ 0.  2.  0. ...  2.  0.  0.]
 [ 0.  0.  2. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  2.]]


In [69]:
# General analysis data:

print("No Samples:\t\t %d" % X.shape[0])
print("No Features:\t\t %d" % X.shape[1])

missing = ((np.isnan(X)).sum()/X.size*100)
print("Missing Values:\t\t %.2f%%" % missing)

No Samples:		 1826
No Features:		 5000
Missing Values:		 10.00%


In [ ]:
# Assess missing values per sample

# Remove samples with > x% missing values

In [ ]:
# Match genotypes with phenotype values

# Only keep values that match

In [ ]:
# Impute missing values